# `jupyterlite` blog integration

the primary goal of this document is define a `doit` task that:
1. build a `jupyterlite` site
2. make the dependencies compatible with `jupyterlite`

In [1]:
    def task_lite():
        """build the jupyter lite site and append requirements"""
        return dict(
            actions=[
                "jupyter lite build --contents tonyfast --output-dir site/run",
                (set_files_imports, (pathlib.Path("site/run/files"),))
            ],
            clean=["rm -rf site/run"]
        )

    import typing, tonyfast, pathlib, textwrap, re, json

`set_files_imports` iterates through a directory and amends notebooks to work in `jupyterlite`

In [2]:
    def set_files_imports(FILES: typing.Iterable[pathlib.Path]=(
        FILES := (WHERE := pathlib.Path(tonyfast.__file__).parent.parent) / "site/run/files"
    )) -> None:
        for file in FILES.rglob("*.ipynb"):  set_file_imports(file)

## building `tonyfast` as `jupyterlite`

some documents might use [pidgy] syntax that need to be dealt with.

`get_imports` finds the imports in each cell

In [3]:
    def get_imports(cell: dict, pidgy=False) -> set:
        import depfinder
        __import__("requests_cache").install_cache()
        source = "".join(cell["source"])
        if pidgy:
            source = tangle.render(source)
        source = textwrap.dedent(source)
        try:
            found = depfinder.inspection.get_imported_libs(source)
            return found.required_modules.union(found.sketchy_modules)
        except BaseException as e:
            return

`get_deps` transforms inputs to dependencies

In [4]:
    def get_deps(deps: set) -> set:
        if "requests" in deps: deps.add("pyodide-http")
        if "pandas" in deps: deps.add("jinja2")
        return {
            x for x in deps 
            if not x.startswith("_") or x not in {"tonyfast"}
        }

In [5]:
    PIDGY = re.compile("^\s*%(re)?load_ext\s*pidgy")
    from midgy import Python; tangle = Python()
    def has_pidgy(nb: dict):
        yes = False
        for _, cell in iter_code_cells(nb):
            yes = yes or PIDGY.match("".join(cell["source"])) and True
        return yes

`set_file_imports` operates in one file discovers dependencies and writes code back to the source.

In [6]:
    def set_file_imports(file: pathlib.Path) -> None:
        data = json.loads(file.read_text())
        deps, first = set(), None
        pidgy = has_pidgy(data)
        for no, cell in iter_code_cells(data):
            if first is None:
                first = no
            deps.update(get_imports(cell, pidgy) or [])
            
        deps = get_deps(deps)
        if deps and (first is not None):
            cell = data["cells"][first]
            was_str = isinstance(cell["source"], str)
            if was_str:
                cell["source"] = cell["source"].splitlines(1)
            
            for i, line in enumerate(list(cell["source"])):
                if (left := line.lstrip()):
                    if left.startswith(("%pip install",)):
                        break
                    indent = len(line) - len(left)                    
                    if "pyodide-http" in deps:
                        data["cells"][first]["source"].insert(i, " "*indent + "__import__('pyodide_http').patch_all()\n")
                    data["cells"][first]["source"].insert(i, " "*indent + "%pip install " + " ".join(deps) +"\n")
                    print(F"writing {len(set(deps))} pip requirements to {file}")
                    file.write_text(json.dumps(data, indent=2))
                    break
        else:
            print(F'no deps for {file}')

`set_files_imports` sets the dependencies for a lot of files.

In [7]:
    def set_files_imports(FILES: typing.Iterable[pathlib.Path]=FILES):
        for file in FILES.rglob("*.ipynb"):
            set_file_imports(file)
            

`iter_code_cells` iterates through just the code cells.

In [8]:
    def iter_code_cells(nb: dict) -> typing.Iterator[tuple[int, dict]]:
        for i, cell in enumerate(nb["cells"]):
            if cell["cell_type"] == "code":
                yield i, cell

## usage

* from the `tonyfast` module, requires deps
* from `hatch` contains deps
* from post with `importnb`

In [9]:
    if '__file__' not in locals():
        !importnb -t 2022-12-21-lite-build.ipynb list

lite   build the jupyter lite site and append requirements
